In [1]:
import yt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
d1 = '/home/u0890475/Downloads/plt59250'

ds = yt.load(d1)
#ad = ds.all_data()

yt : [INFO     ] 2026-02-10 12:00:36,296 Parameters: current_time              = 0.1696800000000185
yt : [INFO     ] 2026-02-10 12:00:36,296 Parameters: domain_dimensions         = [384  64  64]
yt : [INFO     ] 2026-02-10 12:00:36,296 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2026-02-10 12:00:36,297 Parameters: domain_right_edge         = [0.105  0.0175 0.0175]


In [13]:
import numpy as np
import yt


d1 = '/home/u0890475/Downloads/plt59250'

ds = yt.load(d1)
ad = ds.all_data()
ds.force_periodicity()
ds.add_gradient_fields(("boxlib", "Y(H2)"))
# --- pick your temperature field key (adjust if yours is different) ---
Tkey = ('boxlib', 'temp')  # or ("boxlib","temperature")
xkey = ("index", "x")

# -----------------------------
# 1) Region A: T > 400 K and x < 0.07 m
# -----------------------------
reg400 = ad.cut_region([
    f"(obj['{Tkey[0]}','{Tkey[1]}'] > 400)",
    f"(obj['{xkey[0]}','{xkey[1]}'] < 0.07)"
])

# (optional) count how many cells in this region
print("N(T>400 & x<0.07) =", int(reg400[Tkey].size))

# -----------------------------
# 2) Region B (inside Region A): T > 600 K
#    Find min/max of x,y,z in THIS hot region
# -----------------------------
xmin_400 = reg400[("index", "x")].min()
xmax_400 = reg400[("index", "x")].max()
print(xmin_400)
xcut = float(0.5 * (xmin_400 + xmax_400))
print(xcut)
reg600 = reg400.cut_region([
    f"(obj['{Tkey[0]}','{Tkey[1]}'] > 500)",
    f"(obj['index','x'] < {xcut})"
])

# If nothing matches, avoid crashing on min/max
if reg600[Tkey].size == 0:
    raise RuntimeError("No cells found with T > 600 K inside (T > 400 K and x < 0.07 m).")

x = reg600[("index", "x")].to("m").ndarray_view()
y = reg600[("index", "y")].to("m").ndarray_view()
z = reg600[("index", "z")].to("m").ndarray_view()

mins = (x.min(), y.min(), z.min())
maxs = (x.max(), y.max(), z.max())

print("For T > 600 K (within T > 400 K & x < 0.07 m):")
print(f"x_min, x_max = {mins[0]:.6e}, {maxs[0]:.6e} m")
print(f"y_min, y_max = {mins[1]:.6e}, {maxs[1]:.6e} m")
print(f"z_min, z_max = {mins[2]:.6e}, {maxs[2]:.6e} m")


yt : [INFO     ] 2026-02-10 12:04:18,842 Parameters: current_time              = 0.1696800000000185
yt : [INFO     ] 2026-02-10 12:04:18,842 Parameters: domain_dimensions         = [384  64  64]
yt : [INFO     ] 2026-02-10 12:04:18,842 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2026-02-10 12:04:18,842 Parameters: domain_right_edge         = [0.105  0.0175 0.0175]


N(T>400 & x<0.07) = 122741
0.06581298828124998 code_length
0.06789794921874999
For T > 600 K (within T > 400 K & x < 0.07 m):
x_min, x_max = 6.591553e-04, 6.786377e-04 m
y_min, y_max = 6.032715e-05, 8.596191e-05 m
z_min, z_max = 1.708984e-07, 1.469727e-05 m


In [ ]:

# 2) access gradient on your cut region (reg600)
gradH2x = reg600[("boxlib", "Y(H2)_gradient_x")]
gradH2y = reg600[("boxlib", "Y(H2)_gradient_y")]
gradH2z = reg600[("boxlib", "Y(H2)_gradient_z")]
gradH2mag = reg600[("boxlib", "Y(H2)_gradient_magnitude")]

In [31]:
print("length_unit =", ds.length_unit)
print("code_length in m =", ds.quan(1.0, "code_length").to("m"))
print("domain_left_edge =", ds.domain_left_edge)
print("domain_right_edge =", ds.domain_right_edge)
print("domain_width (m) =", ds.domain_width.to("m"))
x = reg600[("index","x")].to_value("m")
y = reg600[("index","y")].to_value("m")
z = reg600[("index","z")].to_value("m")

print("x range (m):", x.min(), x.max())
print("y range (m):", y.min(), y.max())
print("z range (m):", z.min(), z.max())

length_unit = 1.0 cm
code_length in m = 0.01 m
domain_left_edge = [0. 0. 0.] code_length
domain_right_edge = [0.105  0.0175 0.0175] code_length
domain_width (m) = [0.00105  0.000175 0.000175] m


ValueError: zero-size array to reduction operation minimum which has no identity

In [31]:
dx = ds.index.get_smallest_dx()
reg = ds.r[
    ds.domain_left_edge[0] + 8*dx : ds.domain_right_edge[0] - 8*dx,
    ds.domain_left_edge[1] + 8*dx : ds.domain_right_edge[1] - 8*dx,
    ds.domain_left_edge[2] + 8*dx : ds.domain_right_edge[2] - 8*dx,
]

ds.add_gradient_fields(("boxlib","Y(H2)"))
gradH2x = reg[("boxlib","Y(H2)_gradient_x")]

RuntimeError: yt attempted to read outside the boundaries of a non-periodic domain along dimension 0.
Region left edge = -0.0005468749999999999 code_length, Region right edge = 0.004921874999999999 code_length
Dataset left edge = 0.0 code_length, Dataset right edge = 0.105 code_length

This commonly happens when trying to compute ghost cells up to the domain boundary. Two possible solutions are to select a smaller region that does not border domain edge (see https://yt-project.org/docs/analyzing/objects.html?highlight=region)
or override the periodicity with
ds.force_periodicity()

In [ ]:
ds.add_gradient_fields(("boxlib", "Y(H2)"))

# --- 3) Build an interior covering grid WITH ghost zones (avoids boundary issues) ---
dx = ds.index.get_smallest_dx()

ng = 4                    # ghost zones needed by gradient fields
margin = (ng + 8) * dx     # keep grid safely inside domain; increase if needed

le = ds.domain_left_edge + margin
re = ds.domain_right_edge - margin

level = ds.index.max_level  # choose finest AMR level (or set to a specific int)
dims = np.floor((re - le) / dx).astype(int)
dims = np.maximum(dims, 1)

cg = ds.covering_grid(level=level, left_edge=le, dims=dims, num_ghost_zones=ng)

# --- 4) Extract xyz coordinates and gradients ---
x = cg[("index", "x")].to("m").ndarray_view()
y = cg[("index", "y")].to("m").ndarray_view()
z = cg[("index", "z")].to("m").ndarray_view()

gradH2x = cg[("boxlib", "Y(H2)_gradient_x")].ndarray_view()
gradH2y = cg[("boxlib", "Y(H2)_gradient_y")].ndarray_view()
gradH2z = cg[("boxlib", "Y(H2)_gradient_z")].ndarray_view()
gradH2mag = cg[("boxlib", "Y(H2)_gradient_magnitude")].ndarray_view()

# --- 5) Flatten into a table (optional, but convenient) ---
df = pd.DataFrame({
    "x_m": x.ravel(),
    "y_m": y.ravel(),
    "z_m": z.ravel(),
    "dYH2_dx": gradH2x.ravel(),
    "dYH2_dy": gradH2y.ravel(),
    "dYH2_dz": gradH2z.ravel(),
    "|gradYH2|": gradH2mag.ravel(),
})

In [9]:
import numpy as np

dx = ds.index.get_smallest_dx()
ng = 4
margin = (ng + 32) * dx

le = ds.domain_left_edge + margin
re = ds.domain_right_edge - margin

level = ds.index.max_level
dims = np.floor((re - le) / dx).astype(int)

print("level =", level)
print("dims  =", dims, " total cells =", int(np.prod(dims)))

level = 3
dims  = [3000  440  440] dimensionless  total cells = 580800000
